# NeuroFetal AI - Enhanced Fusion ResNet Training

**Version 2.0** - Advanced Enhancements:
- 3-Input Architecture (FHR + Tabular + CSP)
- Squeeze-and-Excitation (SE) Blocks
- Multi-Head Self-Attention
- Focal Loss for Class Imbalance

### Instructions:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run cells in order
3. Use ** Update Repo** cell anytime to pull latest changes

## Step 0: GitHub Authentication

In [26]:
from getpass import getpass
import os

GITHUB_REPO = "Krishna200608/NeuroFetal-AI"
GITHUB_TOKEN = getpass("GitHub Personal Access Token: ")

os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
os.environ['GITHUB_REPO'] = GITHUB_REPO
print("Token saved to session.")

GitHub Personal Access Token: ··········
Token saved to session.


## Step 1: Clone Repository

In [1]:
import os
import shutil

# Always start from /content
os.chdir("/content")

GITHUB_REPO = os.environ.get('GITHUB_REPO', 'Krishna200608/NeuroFetal-AI')
GITHUB_TOKEN = os.environ.get('GITHUB_TOKEN', '')

# Clean clone for fresh start
if os.path.exists("NeuroFetal-AI"):
    shutil.rmtree("NeuroFetal-AI")
    print("Removed old clone.")

print("Cloning repository...")
if GITHUB_TOKEN:
    !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_REPO}.git
else:
    !git clone https://github.com/{GITHUB_REPO}.git

# Verify
dataset_path = "/content/NeuroFetal-AI/Datasets/ctu_uhb_data"
if os.path.exists(dataset_path):
    count = len([f for f in os.listdir(dataset_path) if f.endswith('.dat')])
    print(f"\nSUCCESS: Found {count} dataset records!")
else:
    print(f"\n ERROR: Dataset folder not found!")

Cloning repository...
Cloning into 'NeuroFetal-AI'...
remote: Enumerating objects: 1533, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 1533 (delta 3), reused 2 (delta 1), pack-reused 1516 (from 3)
Receiving objects: 100% (1533/1533), 232.15 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (778/778), done.
Updating files: 100% (1170/1170), done.

SUCCESS: Found 552 dataset records!


## Update Repo (Run anytime to pull latest changes)
**Run this cell whenever you make changes on GitHub or encounter errors.**

In [13]:
import os
os.chdir("/content/NeuroFetal-AI")
# !git checkout -- Code/models/
!git pull origin main
print("\n Repository updated! Re-run training cells below.")

From https://github.com/Krishna200608/NeuroFetal-AI
 * branch            main       -> FETCH_HEAD
Already up to date.

 Repository updated! Re-run training cells below.


In [14]:
# 1. Force hard reset to get latest code
!git fetch origin
!git reset --hard origin/main

HEAD is now at 136ad39 fix: Robust CSP 4-arg fit implementation for proper validation


In [15]:
!cat Code/utils/csp_features.py | grep "def fit"

    def fit(self, X_normal, X_pathological):
    def fit_transform(self, X_normal, X_pathological):
    def fit(self, X_fhr_normal, X_uc_normal, X_fhr_path, X_uc_path):


## Step 2: Setup Paths

In [17]:
import os
import sys

PROJECT_ROOT = "/content/NeuroFetal-AI"
CODE_DIR = os.path.join(PROJECT_ROOT, "Code")
SCRIPTS_DIR = os.path.join(CODE_DIR, "scripts")
UTILS_DIR = os.path.join(CODE_DIR, "utils")

os.chdir(SCRIPTS_DIR)
sys.path.insert(0, SCRIPTS_DIR)
sys.path.insert(0, UTILS_DIR)
sys.path.insert(0, CODE_DIR)

print(f" Working dir: {os.getcwd()}")

 Working dir: /content/NeuroFetal-AI/Code/scripts


## Step 3: Install Dependencies

In [4]:
!pip install -q wfdb shap scipy imbalanced-learn --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 11.5 MB/s eta 0:00:00


In [31]:
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

TensorFlow: 2.19.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Step 4: Data Ingestion

In [32]:
# Cell: Remove old processed data (run once before re-ingestion)
import os

processed_dir = "/content/NeuroFetal-AI/Datasets/processed"
files_to_remove = ["X_fhr.npy", "X_tabular.npy", "y.npy", "X_uc.npy"]

for f in files_to_remove:
    path = os.path.join(processed_dir, f)
    if os.path.exists(path):
        os.remove(path)
        print(f"Removed: {f}")

print("Old data cleared. Ready for re-ingestion.")

Old data cleared. Ready for re-ingestion.


In [18]:
import os
processed_dir = "/content/NeuroFetal-AI/Datasets/processed"

if os.path.exists(os.path.join(processed_dir, "X_fhr.npy")):
    print("Processed data exists. Skipping ingestion.")
else:
    print("Running data ingestion...")
    !python data_ingestion.py

Processed data exists. Skipping ingestion.


In [19]:
# Add
import numpy as np

X_fhr = np.load(os.path.join(processed_dir, "X_fhr.npy"))
X_tabular = np.load(os.path.join(processed_dir, "X_tabular.npy"))
y = np.load(os.path.join(processed_dir, "y.npy"))

print(f"FHR: {X_fhr.shape}, Tabular: {X_tabular.shape}, Labels: {y.shape}")
print(f"Class Balance: {np.mean(y):.2%} positive")

FHR: (2760, 1200), Tabular: (2760, 3), Labels: (2760,)
Class Balance: 7.25% positive


## Step 5: Training

In [16]:
!python train.py

2026-02-05 11:46:15.652543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770291975.672380    6544 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770291975.678356    6544 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770291975.693873    6544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770291975.693898    6544 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770291975.693904    6544 computation_placer.cc:177] computation placer alr

In [20]:
# 3. After training, run uncertainty evaluation
!python evaluate_uncertainty.py

2026-02-05 13:34:07.390389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770298447.410006    3344 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770298447.415982    3344 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770298447.431681    3344 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770298447.431707    3344 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770298447.431711    3344 computation_placer.cc:177] computation placer alr

## Step 6: Explainability (XAI)

In [18]:
!python xai.py

2026-02-05 12:43:30.173298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770295410.193247   29457 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770295410.199421   29457 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770295410.214377   29457 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770295410.214402   29457 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770295410.214406   29457 computation_placer.cc:177] computation placer alr

## Step 7: Push Results to GitHub

In [19]:
!git config --global user.email "krishnasikheriya001@gmail.com"
!git config --global user.name "Krishna200608"

In [20]:
import os
os.chdir("/content/NeuroFetal-AI")

# !git add Code/models/*.keras Reports/training_logs/*.json Code/figures/*.png 2>/dev/null || true
!git add .
!git status
!git commit -m "Add trained model from Colab" || echo "Nothing to commit"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Code/figures/shap_summary.png
	modified:   Code/models/enhanced_model_fold_1.keras
	modified:   Code/models/enhanced_model_fold_2.keras
	modified:   Code/models/enhanced_model_fold_3.keras
	modified:   Code/models/enhanced_model_fold_4.keras
	modified:   Code/models/enhanced_model_fold_5.keras
	new file:   Reports/training_logs/training_log_20260205_124316.json

[main 205c19b] Add trained model from Colab
 7 files changed, 49 insertions(+)
 rewrite Code/figures/shap_summary.png (82%)
 create mode 100644 Reports/training_logs/training_log_20260205_124316.json
Enumerating objects: 26, done.
Counting objects: 100% (26/26), done.
Delta compression using up to 2 threads
Compressing objects: 100% (14/14), done.
Writing objects: 100% (14/14), 99.23 MiB | 10.78 MiB/s, done.
Total 14 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving

## Results

In [21]:
import json, glob, os

log_dir = "/content/NeuroFetal-AI/Reports/training_logs"
logs = sorted(glob.glob(os.path.join(log_dir, "training_log_*.json")))

if logs:
    with open(logs[-1]) as f:
        r = json.load(f)
    print(f"Mean AUC: {r['summary']['mean_auc']:.4f} ± {r['summary']['std_auc']:.4f}")
else:
    print("No training logs found.")

Mean AUC: 0.7731 ± 0.0292
